<a href="https://colab.research.google.com/github/VenkteshV/Project6_7/blob/main/ncert_tagrec%2B_inference_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import logging
logging.basicConfig(level=logging.ERROR)
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu") 


There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/dataEM/ncert_updated.csv")


In [ ]:
data[:6]

,Unnamed: 0,taxonomy,text
0,0,class/9/Science/motion,ffffforceorceorceorceorce andandandandand ...
1,1,class/9/Science/life processes,wwwwworkorkorkorkork andandandandand e e e...
2,2,class/9/Science/gravitation,"in chapters 8 and 9, we have learnt about the ..."
3,3,class/9/Science/motion,"in everyday life, we see some objects at rest ..."
4,4,class/9/Science/water,atter ininininin o o o o oururururur s s s...
5,5,class/9/Science/,iversity ininininin l l l l living iving o...


In [ ]:
data["taxonomy"].values

array(['class/9/Science/motion', 'class/9/Science/life processes',
       'class/9/Science/gravitation', 'class/9/Science/motion',
       'class/9/Science/water', 'class/9/Science/', 'class/9/Science/',
       'class/9/Science/water', 'class/9/Science/water',
       'class/9/Science/structure of the atom', 'class/9/Science/tissues',
       'class/9/Science/water', 'class/9/Science/', 'class/9/Science/',
       'class/9/Science/sound', 'class/8/science/sound',
       'class/8/science/friction', 'class/8/science/force and pressure',
       'class/8/science/materials : metals and non-metals',
       'class/8/science/combustion and flame',
       'class/8/science/synthetic fibres and plastics',
       'class/8/science/microorganisms : friend and foe',
       'class/8/science/',
       'class/8/science/crop production and management',
       'class/8/science/reaching the age of adolescence',
       'class/8/science/gravitation',
       'class/8/science/pollution of air and water',
       'c

In [ ]:
import numpy as np
def clean_taxonomy(data):
    cleaned_tax = []
    data["split_taxonomies"] = data["taxonomy"].apply(lambda x: x.split("/"))
    for taxonomy in  data["split_taxonomies"].values:
        # if 'cbse' in taxonomy:
        #     cbse_index = taxonomy.index('cbse')
        # else:
        #     cbse_index = taxonomy.index('class')
        # if cbse_index!=0:
        #     topic = (' ').join(taxonomy[:cbse_index])
        # rest_of_tax = ('>>').join(taxonomy[cbse_index:])
        cleaned_tax.append('>>'.join(taxonomy))
    data2 = data.assign(cleaned_taxonomy = cleaned_tax)
    # data["cleaned_taxonomy"] = pd.Series(cleaned_tax, dtype=np.str)
    return data2
    



In [ ]:
data  = clean_taxonomy(data)

In [ ]:
!pip install transformers==2.8.0

In [ ]:
!pip install tensorflow==1.13.1
! pip install tensorflow-hub==0.7.0

In [ ]:
import re
def clean_sentence(question):
  # print(question)
  question = re.sub('<[^>]*>', ' ',question)
  question = re.sub(' +', ' ', question)
  question = re.sub('\xa0','',question)
  question = question.rstrip()
  question = re.sub('nan','',question)
  question = re.sub(u'\u2004','',question)
  question = re.sub(u'\u2009','',question)

  # question = question.decode("utf-8")
  # question = question.replace(u'\u200\d*','').encode("utf-8")
  question = re.sub('&nbsp','',question)
  question = re.sub('&ndash','',question)
  question = re.sub('\r','',question)
  question = re.sub('\t','',question)
  question = re.sub('\n',' ',question)

  question = re.sub('MathType@.*','',question)
  question = re.sub('&thinsp','',question)
  question = re.sub('&times','',question)
  question = re.sub('\u200b','',question)
  question = re.sub('&rarr;;;','',question)

  return question

In [ ]:
data["text"] = data["text"].apply(lambda x: clean_sentence(x))
data

,Unnamed: 0,taxonomy,text,split_taxonomies,cleaned_taxonomy
0,0,class/9/Science/motion,ffffforceorceorceorceorce andandandandand l l ...,"[class, 9, Science, motion]",class>>9>>Science>>motion
1,1,class/9/Science/life processes,wwwwworkorkorkorkork andandandandand e e e e e...,"[class, 9, Science, life processes]",class>>9>>Science>>life processes
2,2,class/9/Science/gravitation,"in chapters 8 and 9, we have learnt about the ...","[class, 9, Science, gravitation]",class>>9>>Science>>gravitation
3,3,class/9/Science/motion,"in everyday life, we see some objects at rest ...","[class, 9, Science, motion]",class>>9>>Science>>motion
4,4,class/9/Science/water,atter ininininin o o o o oururururur s s s s s...,"[class, 9, Science, water]",class>>9>>Science>>water
...,...,...,...,...,...
81,81,"class/10/Science/acids, bases and salts","acids, bases and salts you have learnt in your...","[class, 10, Science, acids, bases and salts]","class>>10>>Science>>acids, bases and salts"
82,82,class/10/Science/chemical reactions and equations,“facts are not science — as the dictionary is ...,"[class, 10, Science, chemical reactions and eq...",class>>10>>Science>>chemical reactions and equ...
83,83,class/10/Science/periodic classification of el...,periodic classification of elements in class i...,"[class, 10, Science, periodic classification o...",class>>10>>Science>>periodic classification of...
84,84,class/10/Science/,"carbon and its in the last chapter, we came to...","[class, 10, Science, ]",class>>10>>Science>>


In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
class UseSentenceEmbedding():
    def __init__(self):
        # g = tf.Graph()
        with tf.device('/CPU:0'):
        # We will be feeding 1D tensors of text into the graph.
            self.text_input = tf.placeholder(dtype=tf.string, shape=[None])
            
            #kindly replace the location in hub.module with the url commented out below

            # "https://tfhub.dev/google/universal-sentence-encoder-large/3"
            embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-large/3")
            self.embedded_text = embed(self.text_input)
            init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
        # g.finalize()

        self.session = tf.Session(config=tf.ConfigProto( allow_soft_placement=True))
        self.session.run(init_op)
        print("init _____")



    def get_tokenized_sents_embeddings_USE(self, sents,expand=False):

           
        vectors_USE =  self.session.run(self.embedded_text, feed_dict={self.text_input: sents})

        return vectors_USE

use_embedding = UseSentenceEmbedding()

INFO:absl:hub.KerasLayer is not available because TensorFlow version is less than 1.14
INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-large/3'.
INFO:absl:Downloaded https://tfhub.dev/google/universal-sentence-encoder-large/3, Total size: 810.60MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-large/3'.


init _____


In [ ]:
import numpy as np
!pip install inflection


import inflection

from nltk.stem import PorterStemmer 
ps = PorterStemmer()
from gzip import open as gopen
from pandas.core.common import flatten
import gensim.models.poincare as poincare
def get_cleaned_taxonomy(taxonomy):
  cleaned_taxonomy = []
  for value in taxonomy:
      value = ' '.join(value.split(">>"))
      # taxonomy_words = [inflection.singularize(val)  for token in value for val in token.split(" ") if val.isalpha()]
      cleaned_taxonomy.append( value )
  return cleaned_taxonomy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
qc_science_data = pd.read_csv("/content/drive/MyDrive/dataEM/train_taxonomy_prediction.csv")
label_set = list(set(qc_science_data["board_syllabus"].values))

In [ ]:
len(label_set)

312

In [ ]:
targets = get_cleaned_taxonomy(label_set)
len(targets)

312

In [ ]:
taxonomy_vectors = use_embedding.get_tokenized_sents_embeddings_USE(targets)
taxonomy_vectors.shape

(312, 512)

In [ ]:
train_poincare_tensor = torch.tensor(taxonomy_vectors,dtype=torch.float)

In [ ]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset, random_split


def get_tokenized_input(text):
    input_ids = []
    attention_masks = []

    for sent in text:
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 128,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            truncation=True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                    )
        
        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])
        
        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)


    # Print sentence 0, now as a list of IDs.
    print('Original: ', text[0])
    print('Token IDs:', input_ids[0])
    return input_ids, attention_masks

In [ ]:
input_ids, attention_masks = get_tokenized_input(data["text"].values)

Original:  CBSE NotesCBSE Notes Class 6 ScienceNCERT Solutions Science CBSE Class 6 Science Notes Chapter 2 Components of Food Pdf free download is part of Class 6 Science Notes for Quick Revision. Here we have given NCERT Class 6 Science Notes Chapter 2 Components of Food. CBSE Class 6 Science Notes Chapter 2 Components of Food Food: It is the group of edible substances which provide energy to the living beings and repair the old tissues and build the new tissues. Nutrition: The process of consuming nutrients required for the growth and development of our body and to obtain energy is known as nutrition. Our food contains three main substances called nutrients. These are fats, proteins and carbohydrates. In addition, our body requires water, salts (minerals), vitamins and fibres. Diet: It is the amount of food eaten by a person at a time. Malnutrition: If a person does not get adequate food, or if his/her diet does not contain all the nutrients, he or she becomes weak. When the body do

In [ ]:

import sys
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from matplotlib import pyplot as plt
from torch.nn.modules.loss import HingeEmbeddingLoss
from random import randint

from tqdm import tqdm
import time
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support
from matplotlib import pyplot as plt
from torch.nn.modules.loss import HingeEmbeddingLoss
from random import randint
import torch.nn.functional as F

import time
import argparse
cos = nn.CosineSimilarity(dim=0, eps=1e-6)
cos_label = nn.CosineSimilarity(dim=1, eps=1e-5)

dist = torch.nn.PairwiseDistance(p=2.0, eps=1e-06)
nn.PairwiseDistance(p=2)
class MHSA(nn.Module):
  def __init__(self,
         emb_dim,
         kqv_dim,
         num_heads=2):
    super(MHSA, self).__init__()
    self.emb_dim = emb_dim
    self.kqv_dim = kqv_dim
    self.num_heads = num_heads

    self.w_k = nn.Linear(emb_dim, kqv_dim * num_heads, bias=False)
    self.w_q = nn.Linear(emb_dim, kqv_dim * num_heads, bias=False)
    self.w_v = nn.Linear(emb_dim, kqv_dim * num_heads, bias=False)
    self.w_out = nn.Linear(kqv_dim * num_heads, emb_dim)

  def forward(self, query, key, value):
    # print("query",query.shape)
    b, t = query.shape
    e = self.kqv_dim
    h = self.num_heads
    keys = self.w_k(key).view(b, h, e)
    values = self.w_v(value).view(b, h, e)
    queries = self.w_q(query).view(b, h, e)

    # keys = keys.transpose(2, 1)
    # queries = queries.transpose(2, 1)
    # values = values.transpose(2, 1)

    dot = queries @ keys.transpose(2, 1)  #(b*h*e) @ (b*e*h)
    dot = dot / np.sqrt(e)  # (b*h*h)
    dot = F.softmax(dot, dim=2)

    out = dot @ values   # (b*h*h) @ (b*h*e) = (b*h*e)
    out = out.contiguous().view(b, h * e)
    out = self.w_out(out)
    return out
# Neural Classifierwork

# Discussion TODOS
# try hierarhical interaction (TODO)

# try bringing in modalities (image, or video)

# Go from classical algorithm -> deep learning

class MulticlassClassifier(nn.Module):
    def __init__(self,bert_model_path):
        super(MulticlassClassifier,self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_path,output_hidden_states=True,output_attentions=False)
        self.dropout = nn.Dropout(0.1)
        self.fc1 = nn.Linear(768, 512)
        self.fc2 = nn.Linear(384, 512)
        self.multi_head_attention = MHSA(512, 512,8)
        self.multihead_attn = torch.nn.MultiheadAttention(embed_dim = 512,  num_heads = 8, batch_first=True)


    def forward(self,tokens,masks, targets=None, skip_attention=False):
        # print("tokens", tokens.shape)
        outputs = self.bert(tokens, attention_mask=masks)[2]
        # outputs[2] = outputs[2].permute(0,2,1)
        output_1 = outputs[-1].permute(1,0,2)
        # print(outputs[1].shape,outputs[0].shape)
        output_1 = torch.mean(output_1, dim=0)
        # output_2 = outputs[-2].permute(1,0,2)
        # output_2 = torch.mean(output_2, dim=0)
        # print("output_2", output_2.shape, output_1.shape)
        pooled_output = outputs[-1] #output_1 # torch.cat((output_1, output_2), dim=1)
        # print("pooled_output", pooled_output.shape)
        x = self.fc1(pooled_output)
        # x = self.fc2(x)
        # print("x shape",x.shape)
        targets_curr_batch = []
        for input_x in x:
            # print(input_x.shape)
            distance = cos_label(torch.mean(input_x,dim=0).reshape(1,-1), train_poincare_tensor)
            distances,indices = torch.topk(distance,1,largest=True)

            target_distances = (F.normalize(train_poincare_tensor[indices],p=2,dim=1) - F.normalize(train_poincare_tensor,p=2,dim=1)).pow(2).sum(1) #cos_label(unique_poincare_tensor[indices].reshape(1,-1), unique_poincare_tensor)
            distances,indices = torch.topk(target_distances,5,largest=False)
            targets_curr_batch.append(train_poincare_tensor[indices].reshape(1,5,512))
            # targets_curr_batch.append(unique_poincare_tensor[indices])
            # print("here")
        # print(len(targets_curr_batch))
        targets_batch = torch.cat(targets_curr_batch, dim=0)
        # print("targets_batch",targets_batch.shape)
        attn_output, attn_output_weights = self.multihead_attn(targets_batch, x, x)
        # target_attn_output, attn_output_weights = self.multihead_attn(x, targets_batch, targets_batch)

        x = torch.sum(attn_output,dim=1)
        # print("X shape",x.shape)
        return x

class MyHingeLoss(torch.nn.Module):
    def __init__(self, margin):
        super(MyHingeLoss, self).__init__()
        self.margin = margin

    def forward(self, output, target):
        loss=0
        for i in range(len(output)):
            v_image = F.normalize(output[i],p=2,dim=0)
            t_label = F.normalize(target[i],p=2,dim=0)

            for i in range(5):
                j = randint(0, len(output)-1)
                while j == i:
                    j = randint(0, len(output)-1)
                t_j = F.normalize(target[j],p=2,dim=0)
                loss+= torch.relu( self.margin - cos(t_label, v_image) + cos(t_j, v_image) )
        return loss / (len(output)*5)


In [ ]:
from transformers import BertModel, AdamW, BertConfig
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import BertModel, AdamW, BertConfig

# Loads BertModel, the pretrained BERT model with a single 
model = MulticlassClassifier('bert-base-uncased')
model.load_state_dict(torch.load('/content/drive/MyDrive/model_euclidean_USE_cos_final_attention_V3/model_weights'))

# Tell pytorch to run this model on the GPU.
model.cuda()


MulticlassClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

In [ ]:
def get_inference_taxonomies(input_ids, attention_masks, label_set, train_poincare_tensor):
    model.eval()
    cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

    input_ids = input_ids.to('cuda')
    attention_masks = attention_masks.to('cuda')
    train_poincare_tensor = train_poincare_tensor.to('cuda')
    # Tracking variables 
    predictions , true_labels = [], []
    for input_id,attention_mask in zip(input_ids, attention_masks):
        with torch.no_grad():
            outputs = model(input_id.reshape(1,-1),attention_mask.reshape(1,-1))
            
        distances = cos(outputs,train_poincare_tensor)
        distances,indices = torch.topk(distances,3,largest=True)
        label_set = np.array(label_set)
        predictions.append(label_set[indices.cpu().numpy()])
        print(len(predictions))
    return predictions
    

In [ ]:
train_poincare_tensor = train_poincare_tensor.to(device)
predictions = get_inference_taxonomies(input_ids, attention_masks, label_set, train_poincare_tensor)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
predictions

[array(['science>>components of food', 'science',
        'science>>food-where does it comes from?'], dtype='<U116'),
 array(['science>>is matter around us pure',
        'science>>separation of substances', 'science'], dtype='<U116'),
 array(['science>>fibre to fabric',
        'science>>synthetic fibres and plastics',
        'computer science[c++]>>working with operating system'],
       dtype='<U116'),
 array(['science>>getting to know plants',
        'science>>reproduction in plants',
        'science>>human eye and colourful world'], dtype='<U116'),
 array(['science>>our environment',
        'science>>reaching the age of adolescence',
        'chemistry>>chemistry : part ii>>chemistry in everyday life'],
       dtype='<U116'),
 array(['science>>body movements', 'science>>control and coordination',
        'science>>tissues'], dtype='<U116'),
 array(['science>>the living organisms — characteristics and habitats',
        'science>>reproduction in animals', 'science'], dtype='<U1

In [ ]:
final_predictions = []
for pred in predictions:
    pred_list = pred.tolist()
    predictions_1 = ('*').join(pred_list)
    final_predictions.append(predictions_1)
final_predictions = pd.Series(final_predictions, dtype = np.str)
data["tagrec++_predictions"] = final_predictions
data

In [ ]:
data.to_csv("ncert_tagrec_tagged.csv", index=False)